# Actividad 3 — Sistemas Planetarios y Astrobiología
## Manejo de Datos Meteorológicos desde Marte (MEDA / Perseverance)

**Objetivo:** representar la **presión atmosférica** (Pa) medida por MEDA frente a la **hora local marciana** (LMST) para los soles:

- Sol 182 (verano norte)
- Sol 361 (equinoccio de otoño norte)
- Sol 504 (invierno norte)
- Sol 658 (equinoccio de primavera norte)

**Nota de instrumentación:** MEDA no mide continuamente todo el sol; por eso habrá intervalos sin datos.
Para facilitar la comparación, se calcula un **promedio cada 5 minutos** (ventanas regulares) y se rellenan los huecos con `NaN`.


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


## 1) Archivos de entrada

Subí los 4 CSV al entorno. En Colab suelen quedar en `/content/`.
En este proyecto (sandbox) están en `/mnt/data/`.

El siguiente bloque detecta automáticamente dónde están.


In [ ]:
# Detectar carpeta base (Colab: /content, sandbox: /mnt/data)
CANDIDATES = [Path("/content"), Path("/mnt/data")]
BASE_DIR = next((p for p in CANDIDATES if p.exists()), Path("."))

files = {
    182: BASE_DIR / "WE__0182___________DER_PS__________________P01.csv",
    361: BASE_DIR / "WE__0361___________DER_PS__________________P01.csv",
    504: BASE_DIR / "WE__0504___________DER_PS__________________P01.csv",
    658: BASE_DIR / "WE__0658___________DER_PS__________________P01.csv",
}

# Chequeo rápido: que existan
missing = [sol for sol, p in files.items() if not p.exists()]
if missing:
    raise FileNotFoundError(f"Faltan archivos para soles: {missing}. BASE_DIR={BASE_DIR}")

files


## 2) Inspección rápida de estructura (1 archivo)

Antes de procesar, miramos columnas y ejemplo de valores LMST.


In [ ]:
sol_test = 361
df = pd.read_csv(files[sol_test])
print("Columnas:", list(df.columns))
df[["LMST", "PRESSURE"]].head()


## 3) Función: LMST → horas decimales (0–24)

En estos CSV, `LMST` viene con un prefijo del tipo `00361M`:
- Ejemplo: `00361M00:00:00.528`

La función quita el prefijo y convierte a horas decimales:
- `12:30:00.000` → `12.5`


In [ ]:
def lmst_to_hours(lmst: str) -> float:
    """Convierte LMST (ej: '00361M12:30:00.000') a horas decimales 0–24."""
    if pd.isna(lmst):
        return np.nan

    s = str(lmst).strip()
    # quitar prefijo '00XXXM'
    if "M" in s:
        s = s.split("M", 1)[1]

    h, m, sec = s.split(":")
    return int(h) + int(m)/60 + float(sec)/3600

# Tests visibles (para entender la función)
print(lmst_to_hours("00361M12:30:00.000"))  # esperado: 12.5
print(lmst_to_hours("00182M00:00:00.924"))  # cercano a 0


## 4) Lectura mínima del CSV (solo lo necesario)

Nos quedamos con:
- `LMST` (string)
- `PRESSURE` (Pa)
y agregamos:
- `LMST_hours` (0–24)

También ordenamos por hora.


In [ ]:
def read_meda_pressure(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # Selección mínima
    df = df[["LMST", "PRESSURE"]].copy()

    # Conversión de tiempo
    df["LMST_hours"] = df["LMST"].apply(lmst_to_hours)

    # Limpieza básica
    df = df.dropna(subset=["LMST_hours", "PRESSURE"]).sort_values("LMST_hours").reset_index(drop=True)
    return df

df361 = read_meda_pressure(files[361])
df361.head()


### Verificación rápida (sanity check)

- Rango de LMST_hours debería estar en [0, 24]
- La presión debería estar alrededor de cientos de Pa (Marte ~ 600–900 Pa según estación/altura)


In [ ]:
df361["LMST_hours"].describe(), df361["PRESSURE"].describe()


## 5) Promedio cada 5 minutos (ventanas regulares)

Creamos una grilla de 0 a 24 h en pasos de 5 minutos (576 bins).
En cada ventana:
- calculamos la media
- guardamos cuántos puntos había (`N_POINTS`)
- si no hay puntos, ponemos `NaN` (como pide la consigna)

**Importante:** esto NO interpola ni inventa datos.


In [ ]:
def average_5min(df: pd.DataFrame, sol: int, step_minutes: int = 5) -> pd.DataFrame:
    step = step_minutes / 60  # horas
    bins = np.arange(0, 24, step)  # 576 bins si step=5 min

    rows = []
    for t in bins:
        window = df[(df["LMST_hours"] >= t) & (df["LMST_hours"] < t + step)]
        mean_p = window["PRESSURE"].mean() if len(window) > 0 else np.nan

        rows.append({
            "SOL": sol,
            "LMST": t,
            "PRESSURE_MEAN": mean_p,
            "N_POINTS": int(len(window))
        })

    return pd.DataFrame(rows)

avg361 = average_5min(df361, sol=361)
avg361.head(10)


### ¿Cuántos bins tienen datos vs NaN?


In [ ]:
total_bins = len(avg361)
bins_with_data = (avg361["N_POINTS"] > 0).sum()
bins_without_data = (avg361["N_POINTS"] == 0).sum()

total_bins, bins_with_data, bins_without_data


## 6) Procesar los 4 soles y unir resultados

Esto genera una tabla final `meda_5min` con 576 filas por sol.


In [ ]:
avg_all = []
raw_all = {}

for sol, path in files.items():
    df_raw = read_meda_pressure(path)
    raw_all[sol] = df_raw

    df_avg = average_5min(df_raw, sol=sol)
    avg_all.append(df_avg)

meda_5min = pd.concat(avg_all, ignore_index=True)

meda_5min.groupby("SOL").size()


## 7) Plots

### 7.1 Plot RAW (un sol de ejemplo)


In [ ]:
sol_example = 361
df_raw = raw_all[sol_example]

plt.figure(figsize=(10,4))
plt.scatter(df_raw["LMST_hours"], df_raw["PRESSURE"], s=2)
plt.xlim(0, 24)
plt.xlabel("LMST (horas decimales)")
plt.ylabel("Presión (Pa)")
plt.title(f"MEDA — Sol {sol_example} — datos crudos (raw)")
plt.grid(True)
plt.show()


### 7.2 Plot promediado (4 subplots, colores por estación)

**Colores solicitados:**
- Verano → rojo (Sol 182)
- Otoño → naranja (Sol 361)
- Invierno → azul (Sol 504)
- Primavera → verde (Sol 658)


In [ ]:
seasons = {
    182: "Verano",
    361: "Otoño",
    504: "Invierno",
    658: "Primavera"
}

season_colors = {
    "Verano": "red",
    "Otoño": "orange",
    "Invierno": "blue",
    "Primavera": "green"
}

fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=True)
axes = axes.ravel()

for i, sol in enumerate(files.keys()):
    d = meda_5min[(meda_5min["SOL"] == sol) & (~meda_5min["PRESSURE_MEAN"].isna())]
    season = seasons[sol]
    color = season_colors[season]

    axes[i].plot(d["LMST"], d["PRESSURE_MEAN"], ".", color=color)
    axes[i].set_title(f"Sol {sol} — {season}")
    axes[i].set_xlim(0, 24)
    axes[i].set_ylabel("Presión media (Pa)")
    axes[i].grid(True)

axes[2].set_xlabel("LMST (horas)")
axes[3].set_xlabel("LMST (horas)")

plt.suptitle("MEDA — Presión promediada cada 5 min (Jezero, Perseverance)")
plt.tight_layout()
plt.show()


### 7.3 Plot comparativo (una sola figura, puntos, colores por estación)

Se grafican **solo los bins con datos** (sin unir huecos).


In [ ]:
plt.figure(figsize=(12,5))

for sol in files.keys():
    d = meda_5min[(meda_5min["SOL"] == sol) & (~meda_5min["PRESSURE_MEAN"].isna())]
    season = seasons[sol]
    color = season_colors[season]

    plt.plot(d["LMST"], d["PRESSURE_MEAN"], ".", color=color, label=f"{season} (Sol {sol})")

plt.xlim(0, 24)
plt.xlabel("LMST (horas)")
plt.ylabel("Presión media (Pa)")
plt.title("MEDA — Presión promediada cada 5 min (4 estaciones)")
plt.grid(True)
plt.legend()
plt.show()


## 8) Salida lista para el informe

La tabla `meda_5min` ya contiene, para cada sol y cada bin de 5 minutos:
- `SOL`
- `LMST` (inicio de ventana)
- `PRESSURE_MEAN`
- `N_POINTS`

Podés exportarla si querés adjuntar resultados o reutilizarla en comparación con REMS.


In [ ]:
# Export opcional (descomentar si querés guardarlo)
# meda_5min.to_csv("MEDA_pressure_5min_4sols.csv", index=False)

meda_5min.head()
